#Computer Vision Assignmnet 1
Student Name: Metehan Seyran <br />
Student ID: 150170903 <br /><br />

The directory hierarcy:

- 1
  - cat
    - Cat Pictures
  - Malibu.jpg
  - selfcontrol_part.wav



In [ ]:
import numpy as np
import os
import cv2
import moviepy.editor as mpy
from google.colab.patches import cv2_imshow

In [3]:
background = cv2.imread('1/Malibu.jpg')
bgHeight = background.shape[0]
bgWidth = background.shape[1]
ratio = 360/bgHeight

background = cv2.resize(background, (int(bgWidth * ratio), 360))
print(background.shape)

(360, 926, 3)


In [6]:
def getItem(filename):
  ## Return index inside filename to sort according to numerical index
  _, index = filename.split('_')
  index, _ = index.split('.')
  return int(index)

sortedImages = sorted(os.listdir(mainDir), key=getItem)

##Part 1: Dancing alone

In [7]:
def modifyPartyCats(image):
  ## Combine the background image with the green screened cat images
  image_g_channel = image[:,:,1]
  image_r_channel = image[:,:,0]
  foreground = np.logical_or(image_g_channel < 180, image_r_channel > 150)

  nonzero_x, nonzero_y = np.nonzero(foreground)

  nonzero_cat_values = image[nonzero_x, nonzero_y, :]

  new_frame = background.copy()

  new_frame[nonzero_x, nonzero_y, :] = nonzero_cat_values

  new_frame = new_frame[:, :, [2,1,0]]
  return new_frame

def prepareVideo(images_list, part_name="1"):
  ## Using image list, create the video file with specified name
  clip = mpy.ImageSequenceClip(images_list, fps = 25)
  audio = mpy.AudioFileClip('./1/selfcontrol_part.wav').set_duration(clip.duration)
  clip = clip.set_audio(audio)
  clip.write_videofile("part"+part_name+"_video.mp4", codec='libx264')

In [8]:
image_list = []

for filename in sortedImages:
  image = cv2.imread(mainDir+filename)
  new_image = modifyPartyCats(image)
  image_list.append(new_image)

prepareVideo(image_list)

[MoviePy] >>>> Building video part1_video.mp4
[MoviePy] Writing audio in part1_videoTEMP_MPY_wvf_snd.mp3


100%|██████████| 159/159 [00:00<00:00, 572.01it/s]

[MoviePy] Done.
[MoviePy] Writing video part1_video.mp4



100%|██████████| 181/181 [00:03<00:00, 47.38it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: part1_video.mp4 



##Part 2: Dancing with myself

In [10]:
def combineTwoPartyCats(image):
  # reduce image width to fit the background image
  image = image[:, :463, :]
  #flip the image horizontally
  image2 = np.fliplr(image)
  #combine images together horizontally
  image = np.hstack((image,image2))
  return image

image_list_2_cats = []

for filename in sortedImages:
  #read the image as numpy array
  image = cv2.imread(mainDir+filename)
  #combine two cats side by side
  image = combineTwoPartyCats(image)
  #change greenscreen with background image
  image = modifyPartyCats(image)
  #append the image to list for moviepy
  image_list_2_cats.append(image)

prepareVideo(image_list_2_cats, "2")

[MoviePy] >>>> Building video part2_video.mp4
[MoviePy] Writing audio in part2_videoTEMP_MPY_wvf_snd.mp3


100%|██████████| 159/159 [00:00<00:00, 598.34it/s]

[MoviePy] Done.
[MoviePy] Writing video part2_video.mp4



100%|██████████| 181/181 [00:04<00:00, 38.48it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: part2_video.mp4 



## Part 3: Dancing with my Shadow

In [11]:
def makeDarker(point, threshold):
  #create a lookup with provided threshold
  lookup = np.array([max(i-threshold, 0) for i in range(256)])
  #make the points darker using LUT
  point = lookup[point]
  return point

def combineCatAndShadow(image):
  ## Combining the image of darkened cat and original cat
  image = image[:,:463, :]
  image2 = np.fliplr(image)
  image_g_channel = image2[:,:,1]
  image_r_channel = image2[:,:,0]
  foreground = np.logical_or(image_g_channel < 180, image_r_channel > 150)

  nonzero_x, nonzero_y = np.nonzero(foreground)

  nonzero_cat_values = image2[nonzero_x, nonzero_y, :]
  nonzero_cat_values = np.array(makeDarker(nonzero_cat_values.reshape(-1,), 100)).reshape(-1, 3)

  image2Copy = image2.copy()
  image2Copy[nonzero_x, nonzero_y, :] = nonzero_cat_values
  image = np.hstack([image, image2Copy])
  return image
  
img_list_shadow = []

for filename in sortedImages:
  image = cv2.imread(mainDir+filename)
  cat_with_shadow = combineCatAndShadow(image)
  modified_image = modifyPartyCats(cat_with_shadow)
  img_list_shadow.append(modified_image)

prepareVideo(img_list_shadow, "3")

[MoviePy] >>>> Building video part3_video.mp4
[MoviePy] Writing audio in part3_videoTEMP_MPY_wvf_snd.mp3


100%|██████████| 159/159 [00:00<00:00, 564.59it/s]

[MoviePy] Done.
[MoviePy] Writing video part3_video.mp4



100%|██████████| 181/181 [00:04<00:00, 39.02it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: part3_video.mp4 



##Part 3: Dancing with my friend

In [ ]:
#number of cat images
m = len(sortedImages)
#array for average histogram
hist = np.zeros((256, 3))

def getCatPixels(image):
  #Returns the pixels belonging to cat for given image
  image_g_channel = image[:,:,1]
  image_r_channel = image[:,:,0]
  foreground = np.logical_or(image_g_channel < 180, image_r_channel > 150)

  nonzero_x, nonzero_y = np.nonzero(foreground)

  nonzero_cat_values = image[nonzero_x, nonzero_y, :]
  return nonzero_cat_values, nonzero_x, nonzero_y

def calculateHistogram(values):
  ## calculating the histogram by using the values of pixels provided.
  hist = np.zeros([256, 3])

  for g in range(256):
    hist[g, ...] += np.sum((values == g)*1.0, 0)
  
  return hist

for filename in sortedImages:
  image = cv2.imread(mainDir+filename)
  catPixels, _, _ = getCatPixels(image)
  hist += calculateHistogram(catPixels)
  print(filename+" done...")

#averaging the histogram counts
hist = hist//m
#calculate the target (background) image's histogram
target_hist = calculateHistogram(background.reshape(-1, 3))

In [13]:
#Computing CDFs
target_cum = np.cumsum(target_hist, axis=0)
target_cdf = target_cum/np.max(target_cum)

source_cum = np.cumsum(hist, axis=0)
source_cdf = source_cum/np.max(source_cum)

In [14]:
def HistogramUsingLookUp(target, source):
  ## Calculate the LookUp Table using target and source CDFs.
  ## The function is for 3 channel images.
  LUT = np.zeros((256, 3))
  for i in range(3):
    gJ = 0
    for gI in range(256):
      while source[gI, i] > target[gJ, i] and gJ < 255:
        gJ += 1
      LUT[gI, i] = gJ
  return LUT      

LookUpTable = HistogramUsingLookUp(target_cdf, source_cdf)

In [15]:
def combineCatAndFriend(image):
  ## Combining the histogram matched cat with the original cat.
  image = image[:,:463, :]
  image2 = np.fliplr(image)
  image_g_channel = image2[:,:,1]
  image_r_channel = image2[:,:,0]
  foreground = np.logical_or(image_g_channel < 180, image_r_channel > 150)

  nonzero_x, nonzero_y = np.nonzero(foreground)

  nonzero_cat_values = image2[nonzero_x, nonzero_y, :]
  R = LookUpTable[nonzero_cat_values[:, 0], 0].reshape(-1, 1)
  G = LookUpTable[nonzero_cat_values[:, 1], 1].reshape(-1, 1)
  B = LookUpTable[nonzero_cat_values[:, 2], 2].reshape(-1, 1)
  newCatPixels = np.hstack([R, G, B])

  image2Copy = image2.copy()
  image2Copy[nonzero_x, nonzero_y, :] = newCatPixels
  image = np.hstack([image, image2Copy])
  return image

image_list_with_friend = []
for filename in sortedImages:
  image = cv2.imread(mainDir+filename)
  new_image = combineCatAndFriend(image)
  image = modifyPartyCats(new_image)
  image_list_with_friend.append(image)

prepareVideo(image_list_with_friend, "3_2")

[MoviePy] >>>> Building video part3_2_video.mp4
[MoviePy] Writing audio in part3_2_videoTEMP_MPY_wvf_snd.mp3


100%|██████████| 159/159 [00:00<00:00, 594.36it/s]

[MoviePy] Done.
[MoviePy] Writing video part3_2_video.mp4



100%|██████████| 181/181 [00:04<00:00, 38.76it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: part3_2_video.mp4 



##Part 4: Disco dancing

In [ ]:
perturbation = np.random.rand(256, 3)
perturbation

In [17]:
def perturbLeftCat(image):
  ## Perturbation and histogram matching for the left cat

  #obtaining the cat pixels and calculating the histogram
  catPixels, nonzero_x, nonzero_y = getCatPixels(image)
  catHist = calculateHistogram(catPixels)
  ##calculating the Cumulative Histogram and CDF
  catHistCum = np.cumsum(catHist, axis=0)
  catCDF = catHistCum/np.max(catHistCum)
  ##Calculating the target image CDF (target is perturbed histogram of the cat)
  targetHist = perturbation * catHist
  targetHistCum = np.cumsum(targetHist, axis=0)
  targetCDF = targetHistCum/np.max(targetHistCum)
  ##create a lookup table
  LUT = HistogramUsingLookUp(targetCDF, catCDF)
  ##process the new image
  new_image = image.copy()
  R = LUT[catPixels[:, 0], 0].reshape(-1, 1)
  G = LUT[catPixels[:, 1], 1].reshape(-1, 1)
  B = LUT[catPixels[:, 2], 2].reshape(-1, 1)
  new_image[nonzero_x, nonzero_y, :] = np.hstack([R, G, B])

  return new_image

def perturbRightCat(image):
  ## Perturbation and histogram matcing for the right cat
  catPixels, nonzero_x, nonzero_y = getCatPixels(image)
  catHist = calculateHistogram(catPixels)
  ##calculating the Cumulative Histogram and CDF
  catHistCum = np.cumsum(catHist, axis=0)
  catCDF = catHistCum/np.max(catHistCum)
  ##Calculating the target image CDF (target is perturbed histogram of the background)
  targetHist = calculateHistogram(background.reshape(-1, 3)) * perturbation
  targetHistCum = np.cumsum(targetHist, axis=0)
  targetCDF = targetHistCum/np.max(targetHistCum)
  ##create a lookup table 
  LUT = HistogramUsingLookUp(targetCDF, catCDF)
  ##process the new image
  new_image = image.copy()
  R = LUT[catPixels[:, 0], 0].reshape(-1, 1)
  G = LUT[catPixels[:, 1], 1].reshape(-1, 1)
  B = LUT[catPixels[:, 2], 2].reshape(-1, 1)
  new_image[nonzero_x, nonzero_y, :] = np.hstack([R, G, B])

  return new_image


In [ ]:
def combineColorfulCats(image):
  ## Combining new left and right cats.
  image = image = image[:,:463, :]
  image2 = np.fliplr(image)

  leftCat = perturbLeftCat(image)
  rightCat = perturbRightCat(image2)

  result = np.hstack([leftCat, rightCat])
  final_image = modifyPartyCats(result)

  return final_image

image_list_colorful_cats = []

for filename in sortedImages:
  image = cv2.imread(mainDir+filename)
  new_image = combineColorfulCats(image)
  image_list_colorful_cats.append(new_image)
  print(filename+" done...")
  
prepareVideo(image_list_colorful_cats, "4")